In [1]:
import pandas as pd
import os

In [3]:
files = os.listdir("api_data/processed")
df_list = []

FileNotFoundError: [Errno 2] No such file or directory: 'api_data/processed'

In [ ]:
for file in files:
    path = "api_data/processed/" + file
    df_list.append(pd.read_csv(path).drop(columns="Unnamed: 0"))

In [4]:
df = pd.concat(df_list, axis=0, ignore_index=True, sort=False)

NameError: name 'df_list' is not defined

In [5]:
df.to_csv("data.csv")

NameError: name 'df' is not defined

In [102]:
def generateTargets(df):
    df["Target"] = df["Stock Price"].shift(-1)
    return df[:-1]

In [103]:
df = df.dropna(subset=["Stock Price", "Company", "Date"])
df = df[df["Stock Price"] > 0]

In [104]:
df = df.sort_values(["Company", "Date"])

In [105]:
df1 = df.groupby("Company").apply(
    lambda group: generateTargets(group)
)
df1 = df1.droplevel(0)

In [106]:
df1 = df1.dropna()

In [113]:
df1.to_csv("final_cleaned_data.csv")

In [297]:
df1["Test"] = 100*(df1["Target"] - df1["Stock Price"])/df1["Stock Price"]

In [298]:
# Remove volatile stocks
df1 = df1[(df1["Test"] <= 100) & (df1["Test"] >= -100)]

In [344]:
# t = df1[df1["Date"] < "2018"]
t = pd.read_csv("train_raw.csv")

In [345]:
len(t)/9

6951.888888888889

In [348]:
# df1['Label'] = 1
# df1.loc[df1['Test'] > 5, 'Label'] = 2
# df1.loc[df1['Test'] < -5, 'Label'] = 0

# ranges = [-100, -15.5, -7.8, -3, 1.5, 2, 5, 10, 25, 50, 100]
ranges = [-100, -15.5, -7.75, -3, 0.7, 4.3, 8.25, 13.1, 21.5, 100]

for i in range(0, 10):
    print(len(t[t["Label"] == i]))

# for i in range(len(ranges)-1):
#     lower = ranges[i]
#     upper = ranges[i+1]
    
#     l = len(t[(t["Test"] >= lower) & (t["Test"] <= upper)])
    
#     print(lower, "to", upper,":",l)

# holdLabel = 4
# df1['Label'] = holdLabel

# for i in range(len(ranges)-1):
#     lower = ranges[i]
#     upper = ranges[i+1]
    
#     df1.loc[(df1["Test"] >= lower) & (df1["Test"] <= upper), 'Label'] = i

6950
6975
6961
6954
6941
6980
6996
6940
6870
0


In [338]:
df1 = df1.drop("Test", axis=1)

In [340]:
def trainTestValSplit(df, tag):
    test = df[df["Date"] >= "2018"].reset_index(drop=True)
    
    other = df[df["Date"] <= "2018"].reset_index(drop=True)
    # Shuffle
    other = other.sample(frac=1).reset_index(drop=True)
    
    length = len(other)

    train_ratio = 0.95
    train_samples = (int)(length*train_ratio)
    
    train = other[0:train_samples]
    val = other[train_samples:]
    
    train.to_csv("train_" + tag + ".csv", index=False)
    val.to_csv("val_" + tag + ".csv", index=False)
    test.to_csv("test_" + tag + ".csv", index=False)

In [341]:
trainTestValSplit(df1, "raw")

In [342]:
def normalize(df):
    normalized = pd.DataFrame()
    norm_ratios = dict()
    
    for k in list(df.keys()):
        if k != "Company" and k != "Date" and k != "Label" and k != "Target":
            col = df[k]
            mean = col.mean()
            std = col.std()

            normalized[k] = ((col - mean)/std)
            norm_ratios[k] = dict()
            norm_ratios[k]["mean"] = mean
            norm_ratios[k]["std"] = std
        else:
            normalized[k] = df[k]
    
    trainTestValSplit(normalized, "normalized")
    pd.DataFrame(nr).to_csv("norm_ratios.csv")

In [343]:
normalize(df1)

In [234]:
# normalized

In [64]:
# # pd.DataFrame(norm_ratios)
# # norm_ratios

# pd.DataFrame(norm_ratios, index=[0]).to_csv("norm_ratios.csv", index=False)
# normalized.reset_index(drop=True).to_csv("normalized_data.csv", index=False)

In [1]:
import pandas as pd

pd.read_csv("test_raw.csv")

,Company,Date,Revenue,Revenue Growth,Cost of Revenue,Gross Profit,R&D Expenses,SG&A Expense,Operating Expenses,Operating Income,...,Inventory Growth,Asset Growth,Book Value per Share Growth,Debt Growth,R&D Expense Growth,SG&A Expenses Growth,Stock Price,DCF,Target,Label
0,A,2018-01-31,1.211000e+09,0.018503,541000000.0,670000000.0,94000000.0,347000000.0,441000000.0,229000000.0,...,0.1034,0.1049,0.0491,0.0768,0.1899,0.2007,67.51,76.447582,62.69,2
1,A,2018-04-30,1.206000e+09,-0.004129,563000000.0,643000000.0,92000000.0,341000000.0,433000000.0,210000000.0,...,0.0839,0.0958,0.0518,0.0352,0.0952,0.1107,62.69,72.027112,67.26,5
2,A,2018-07-31,1.203000e+09,-0.002488,544000000.0,659000000.0,97000000.0,341000000.0,438000000.0,221000000.0,...,0.1007,0.0107,-0.0065,-0.1355,0.1149,0.1071,67.26,73.920371,72.35,5
3,A,2018-10-31,1.294000e+09,0.075644,585000000.0,709000000.0,104000000.0,356000000.0,460000000.0,249000000.0,...,0.1096,0.0136,-0.0427,-0.1054,0.1685,0.0954,72.35,79.398979,81.24,6
4,A,2019-01-31,1.284000e+09,-0.007728,577000000.0,707000000.0,102000000.0,355000000.0,457000000.0,250000000.0,...,0.0740,0.0292,0.1311,-0.1618,0.0851,0.0231,81.24,87.697729,66.89,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17066,ZYXI,2018-06-30,7.573000e+06,0.101367,1330000.0,6243000.0,0.0,3528000.0,3528000.0,2715000.0,...,1.9101,2.2357,5.7755,-0.9100,0.0000,0.6897,3.04,3.240958,3.05,3
17067,ZYXI,2018-09-30,8.131000e+06,0.073683,1641000.0,6490000.0,0.0,3670000.0,3670000.0,2820000.0,...,1.2235,1.2221,4.5686,-0.9376,0.0000,0.4460,3.05,3.307397,3.85,8
17068,ZYXI,2018-12-31,9.337000e+06,0.148321,1831000.0,7506000.0,0.0,4626000.0,4626000.0,2880000.0,...,0.9787,1.1560,0.9145,9.3446,0.0000,0.5353,3.85,4.173516,5.57,8
17069,ZYXI,2019-03-31,9.196000e+06,-0.015101,1784000.0,7412000.0,0.0,5156000.0,5156000.0,2256000.0,...,0.3634,1.0265,1.2270,33.9700,0.0000,0.3988,5.57,5.862201,8.39,8


In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("val_raw.csv")

In [3]:
list(df.keys())

['Company',
 'Date',
 'Revenue',
 'Revenue Growth',
 'Cost of Revenue',
 'Gross Profit',
 'R&D Expenses',
 'SG&A Expense',
 'Operating Expenses',
 'Operating Income',
 'Interest Expense',
 'Earnings before Tax',
 'Income Tax Expense',
 'Net Income - Non-Controlling int',
 'Net Income - Discontinued ops',
 'Net Income',
 'Preferred Dividends',
 'Net Income Com',
 'EPS',
 'EPS Diluted',
 'Weighted Average Shs Out',
 'Weighted Average Shs Out (Dil)',
 'Dividend per Share',
 'Gross Margin',
 'EBITDA Margin',
 'EBIT Margin',
 'Profit Margin',
 'Free Cash Flow margin',
 'EBITDA',
 'EBIT',
 'Consolidated Income',
 'Earnings Before Tax Margin',
 'Net Profit Margin',
 'Cash and cash equivalents',
 'Short-term investments',
 'Cash and short-term investments',
 'Receivables',
 'Inventories',
 'Total current assets',
 'Property Plant & Equipment Net',
 'Goodwill and Intangible Assets',
 'Long-term investments',
 'Tax assets',
 'Total non-current assets',
 'Total assets',
 'Payables',
 'Short-term 

In [9]:
keys = [
 'Revenue Growth',
 'EPS',
 'EPS Diluted',
 'Dividend per Share',
 'Gross Margin',
 'EBITDA Margin',
 'EBIT Margin',
 'Profit Margin',
 'Free Cash Flow margin',
 'Earnings Before Tax Margin',
 'Net Profit Margin',
 'Revenue per Share',
 'Net Income per Share',
 'Operating Cash Flow per Share',
 'Free Cash Flow per Share',
 'Cash per Share',
 'Book Value per Share',
 'Tangible Book Value per Share',
 'Shareholders Equity per Share',
 'Interest Debt per Share',
 'PE ratio',
 'Price to Sales Ratio',
 'POCF ratio',
 'PFCF ratio',
 'PB ratio',
 'PTB ratio',
 'EV to Sales',
 'Enterprise Value over EBITDA',
 'EV to Operating cash flow',
 'EV to Free cash flow',
 'Earnings Yield',
 'Free Cash Flow Yield',
 'Debt to Equity',
 'Debt to Assets',
 'Net Debt to EBITDA',
 'Current ratio',
 'Interest Coverage',
 'Income Quality',
 'Dividend Yield',
 'Payout Ratio',
 'SG&A to Revenue',
 'R&D to Revenue',
 'Intangibles to Total Assets',
 'Capex to Operating Cash Flow',
 'Capex to Revenue',
 'Capex to Depreciation',
 'Stock-based compensation to Revenue',
 'Graham Number',
 'Graham Net-Net',
 'Working Capital',
 'Tangible Asset Value',
 'Net Current Asset Value',
 'Invested Capital',
 'Average Receivables',
 'Average Payables',
 'Average Inventory',
 'Capex per Share',
 'Gross Profit Growth',
 'EBIT Growth',
 'Operating Income Growth',
 'Net Income Growth',
 'EPS Growth',
 'EPS Diluted Growth',
 'Weighted Average Shares Growth',
 'Weighted Average Shares Diluted Growth',
 'Dividends per Share Growth',
 'Operating Cash Flow growth',
 'Free Cash Flow growth',
 'Receivables growth',
 'Inventory Growth',
 'Asset Growth',
 'Book Value per Share Growth',
 'Debt Growth',
 'R&D Expense Growth',
 'SG&A Expenses Growth',
 'Stock Price',
 'DCF',
]

In [10]:
pd.set_option("display.max_columns", 0)
df[keys]

,Revenue Growth,EPS,EPS Diluted,Dividend per Share,Gross Margin,EBITDA Margin,EBIT Margin,Profit Margin,Free Cash Flow margin,Earnings Before Tax Margin,Net Profit Margin,Revenue per Share,Net Income per Share,Operating Cash Flow per Share,Free Cash Flow per Share,Cash per Share,Book Value per Share,Tangible Book Value per Share,Shareholders Equity per Share,Interest Debt per Share,PE ratio,Price to Sales Ratio,POCF ratio,PFCF ratio,PB ratio,PTB ratio,EV to Sales,Enterprise Value over EBITDA,EV to Operating cash flow,EV to Free cash flow,Earnings Yield,Free Cash Flow Yield,Debt to Equity,Debt to Assets,Net Debt to EBITDA,Current ratio,Interest Coverage,Income Quality,Dividend Yield,Payout Ratio,SG&A to Revenue,R&D to Revenue,Intangibles to Total Assets,Capex to Operating Cash Flow,Capex to Revenue,Capex to Depreciation,Stock-based compensation to Revenue,Graham Number,Graham Net-Net,Working Capital,Tangible Asset Value,Net Current Asset Value,Invested Capital,Average Receivables,Average Payables,Average Inventory,Capex per Share,Gross Profit Growth,EBIT Growth,Operating Income Growth,Net Income Growth,EPS Growth,EPS Diluted Growth,Weighted Average Shares Growth,Weighted Average Shares Diluted Growth,Dividends per Share Growth,Operating Cash Flow growth,Free Cash Flow growth,Receivables growth,Inventory Growth,Asset Growth,Book Value per Share Growth,Debt Growth,R&D Expense Growth,SG&A Expenses Growth,Stock Price,DCF
0,0.130966,0.320,0.31,0.106180,0.107631,0.091965,0.063730,0.034933,0.020524,0.050058,0.034933,9.214705,0.321897,0.338419,0.189124,1.591568,12.901618,21.742234,12.901618,8.164811,53.371032,1.864411,50.765460,90.840059,1.331616,1.331616,2.563235,27.871795,69.793521,124.889000,0.018737,0.011008,1.065361,0.515823,7.598789,3.687747,4.389517,1.051326,0.006651,0.354978,0.043902,0.043902,0.184049,2.266774,61.721268,1.742739,0.002271,9.666564,-6.027560,1.916963e+09,1.796327e+09,-4.571190e+08,5.598348e+09,9.179775e+08,470835000.0,1.543946e+09,0.149295,0.364149,0.629825,0.629825,0.874154,0.882353,0.823529,0.007192,0.001355,0.039474,-3.789812,-1.815543,0.120667,-0.008430,0.012064,0.0000,-0.140494,0.000000,0.103118,21.48,23.071568
1,-0.231387,0.120,0.12,0.000000,0.918267,0.248139,0.158023,0.035413,-0.034003,0.051292,0.035413,3.278028,0.116084,0.029392,-0.111461,1.048197,3.371514,13.230173,3.371514,17.308878,81.148038,2.873679,320.496164,-84.513552,2.793997,2.793997,7.739269,31.189270,863.146493,-227.608267,0.012323,-0.011832,6.454674,0.865856,19.608337,1.891435,1.664847,0.253195,0.000000,-0.000000,0.176684,0.176684,0.473605,0.208670,23.272630,2.097237,0.002191,2.967496,-20.489313,6.237100e+07,-3.968110e+08,-8.798000e+08,1.230063e+09,2.864800e+07,31522000.0,1.242450e+07,0.140853,-0.248752,0.044798,0.044798,0.125965,0.200000,0.200000,0.000079,0.002479,0.000000,-0.945401,-1.353398,0.075367,0.030935,-0.000050,0.0000,0.022230,0.000000,-0.000631,9.90,10.948197
2,0.278688,0.200,0.20,0.100000,0.465600,0.233000,0.213400,0.136000,0.255000,0.213400,0.136400,1.459962,0.199112,0.398999,0.372356,0.621060,2.444249,2.947259,2.444249,0.000000,43.693917,5.959058,21.804541,23.364711,3.559376,3.559376,5.533663,23.782826,20.247997,21.696792,0.022886,0.042800,0.284331,0.221385,-1.828281,3.710986,0.000000,2.003891,0.011492,0.502146,0.126962,0.126962,0.061151,-14.975744,-54.797118,-1.057680,0.009086,3.309125,0.764392,8.930388e+06,1.357437e+07,8.035959e+06,1.072570e+07,4.152348e+06,1002152.0,3.648856e+06,-0.026643,0.122800,0.245900,0.244600,0.154300,0.176500,0.250000,0.006500,0.005600,0.000000,0.105600,0.070200,0.046800,-0.192700,0.123400,0.1049,0.000000,0.000000,0.006400,9.25,9.886169
3,-0.014151,1.400,1.39,0.260000,0.264908,0.149726,0.109698,0.072587,0.082623,0.094760,0.072587,19.170022,1.391499,1.926174,1.583893,4.174497,50.762864,51.597315,50.762864,27.469799,54.186174,3.933224,39.144948,47.604237,1.485338,1.485338,5.132664,34.280436,51.082230,62.121186,0.018455,0.021007,0.967300,0.491689,8.010912,2.218261,6.962963,1.384244,0.003471,0.188103,0.0000